In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
X = 1000
N = 10
x = np.linspace(0,X,N)
y1 = 5 * x * x
y2 = 5 * x * x - 4 * x
ea= y1 - y2
error = y1[-1]-y2[-1]
print('Mesh size :', X * X)
print(100 * error / np.max(y1))
print(error)
print('Kbytes', error * 8 * 8 / 1024)
plt.plot(x, y1,'-o')
plt.plot(x, y2,'-s')
plt.plot(x, ea,'-v')
plt.grid()

In [ ]:
y3 = 7 * x * x * x
y4 = 7 * x * x * x - 6 * x * x
ea = y3 - y4
error = y3[-1]-y4[-1]
print('Mesh size :', X * X * X)
print(100 * error / np.max(y3))
print(error)
print('Mbytes', error * 8 * 8 / (1024 * 1024))
plt.plot(x, y3,'-o')
plt.plot(x, y4,'-s')
plt.plot(x, ea,'-v')
plt.grid()

In [ ]:
def aS(i,j):
    return 1 #-2

def aW (i,j):
    return 1 #-1

def aP(i,j):
    return 1 #5

def aE(i,j):
    return 1

def aN(i,j):
    return 1 #2

In [ ]:
Nx = 3
Ny = 4

Nd = 5

N = Nx * Ny
Nzeros = Nd * N

AA = np.zeros(Nzeros)
JA = np.zeros(Nzeros, dtype=int)
IA = np.zeros(N+1, dtype=int)

In [ ]:
bi = 1
ei = Nx
bj = 1
ej = Ny

In [ ]:
#
# Compressed Sparse Row (CSR) or Compressed Row Storage (CRS)
#
I = 0
irow = 0
IA[irow] = I

for j in range(bi,ei+1):
    for i in range(bj,ej+1):
        
        # S
        AA[I] = aS(i,j)
        JA[I] = irow - Nx
        
        # W
        I += 1 
        AA[I] = aW(i,j)
        JA[I] = irow - 1
        
        # P
        I += 1 
        AA[I] = aP(i,j)
        JA[I] = irow
        
        # E
        I += 1
        AA[I] = aE(i,j)
        JA[I] = irow + 1
        
        #N
        I += 1
        AA[I] = aN(i,j)
        JA[I] = irow + Nx
        
        #######
        I += 1
        irow += 1
        IA[irow] = I

#-----------+
# * * P E N |
# * W P E N |
# * W P E N |
# ...       
# * W P E N |
# * W P * N |
#-----------+
I = 0 # S primero
for i in range(bi,ei+1):
    AA[IA[I]] = 0
    JA[IA[I]] = 0
    I += 1
AA[1] = 0 # W segundo
JA[1] = 0 # W segundo

#-----------+
# S * P E * |
# S W P E * |
# S W P E * |
# ...       
# S W P E * |
# S W P * * |
#-----------+
I = IA[-1]-1 # N ultimo
for i in range(bi,ei+1):
    AA[I] = 0
    JA[I] = N-1
    I -= Nd
AA[-2] = 0 # E penultimo
JA[-2] = N-1 # E penuñtimo


In [ ]:
I = 0
print('-'* 45)
print('|{:>3} | {:>5} | {:>5} {:>5} {:>5} {:>5} {:>5} |'.format('I','IA','S','W','P','E','N'))
print('-'* 45)
for i in range(0,N):
    print('|{:>3} | {:>5} | {:>5} {:>5} {:>5} {:>5} {:>5} |'.format(i, 'AA ->', AA[I], AA[I+1], AA[I+2], AA[I+3], AA[I+4]))
    print('|{:>3} | IA{:>3} | {:>5} {:>5} {:>5} {:>5} {:>5} |'.format(i, IA[i], JA[I], JA[I+1], JA[I+2], JA[I+3], JA[I+4]))
    print('-'*45)
    I += Nd
    

In [ ]:
def matvec(val, col_ind, row_ptr, x):
    N = len(row_ptr)-1
    y = np.zeros(N)
    for i in range(0,N):
        y[i] = 0
        for j in range(row_ptr[i],row_ptr[i+1]):
            y[i] = y[i] + val[j] * x[col_ind[j]]
    return y

In [ ]:
vec = np.ones(N)
matvec(AA,JA,IA,vec)